In [2]:
import sagemaker
import boto3
from sklearn.model_selection import train_test_split
import pandas as pd

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\Lenovo\AppData\Local\sagemaker\sagemaker\config.yaml


In [5]:
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = "sagemakermobilepricepredictionbucket"
print("Using bucket " + bucket + "in region " + region)

Using bucket sagemakermobilepricepredictionbucketin region eu-north-1
